This script reads "processed input data", which is the input to the ML/DNN models, to examine the distributions of input data. 

Here I made the histogram plots:
* original data
* log of original data
* pre-processsing (quantileTransformer) of original data

Under ./data_hist directory, it generates a pdf file for the 10 observation sites (see the do loop with file_ls[0:10]).

In [ ]:
import os, os.path, sys, glob
import pandas
import datetime
import re
import argparse
import logging
import pytz
import tzwhere.tzwhere as tzwhere
import difflib
import numpy as np
import lime
import lime.lime_tabular
import sklearn.preprocessing
import scipy.stats
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

sys.path.append('../code-air-forecast')

import forecast.convertfmt
import forecast.loadcsv
import forecast.fileio
import forecast.modelsk
import forecast.modelsnn
import forecast.modelsln
import forecast.modelfit
import forecast.evaluations

# testing Doris' evaluation code as there is some error

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


input_dir="./processed_data/"
species="pm"
saveprefix = "input*"+species+"*csv"



file_ls = forecast.fileio.get_files(input_dir, pattern=saveprefix)

print(file_ls[0:3])


In [ ]:
output_dir="./data_hist/"

for fifn in file_ls[0:10]:
    d = forecast.fileio.load_csv(fifn)
    
    remove_ls = ['lastday8havg', 'Water_vap_mixing_ratio_kg_per_kg', 'WindDir_deg']
    d.drop([x for x in remove_ls if x in d.columns], axis=1, inplace=True)
        
        
    features = list(d.keys())

    features.remove('index')
    features.remove('date')
    features.remove('weekday')
    features.remove('month')
    features.remove('day')
    features.remove('hour')
    

    
    print(len(features), features)

        
    fn=fifn.split("-")
    filename=output_dir+fn[2]+"-histogram-"+species+".pdf"

    pdf = PdfPages(filename)
     
    fig, axs = plt.subplots(5,4, figsize=(24, 15))
    i=0
    num_bins =30
    
    # unpack all the axes subplots
    axs = axs.ravel()

    for fi in features:  #target:
                            
        dfp_feature= d[fi].apply(pandas.Series)
        print("index", fi)

        dfp_feature.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel(fi)
        
        dfp_log= np.log10(dfp_feature) 
        dfp_log.replace([np.inf, -np.inf], np.nan, inplace=True)
        i=i+1
        #print("index", fi, d[fi])
        dfp_log.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel("log "+fi)
        
        i=i+1
        
    pdf.savefig() 
            
    pdf.close()
    plt.close()
    
    

In [11]:
output_dir="./data_hist/"
from sklearn import preprocessing

quantile_transformer = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
    
    
for fifn in file_ls[0:10]:
    d = forecast.fileio.load_csv(fifn)
    
    remove_ls = ['lastday8havg', 'Water_vap_mixing_ratio_kg_per_kg', 'WindDir_deg']
    d.drop([x for x in remove_ls if x in d.columns], axis=1, inplace=True)
        
        
    features = list(d.keys())

    features.remove('index')
    features.remove('date')
    features.remove('weekday')
    features.remove('month')
    features.remove('day')
    features.remove('hour')
    
    print(len(features), features)

        
    fn=fifn.split("-")
    filename=output_dir+fn[2]+"-histogram-"+species+".pdf"

    pdf = PdfPages(filename)
     
    fig, axs = plt.subplots(5,4, figsize=(24, 15))
    i=0
    num_bins =30
    
    # unpack all the axes subplots
    axs = axs.ravel()

    for fi in features:  #target:
                            
        dfp_feature= d[fi].apply(pandas.Series)
        print("index", fi)

        dfp_feature.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel(fi)
        i=i+1
        
        X_trans = quantile_transformer.fit_transform(dfp_feature)
        df = pd.DataFrame(data=X_trans)

        print(type(X_trans), X_trans.shape)
        df.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel("preprocessing "+fi)
        i=i+1
        
    pdf.savefig() 
            
    pdf.close()
    plt.close()
    

7 ['PBL_m', 'Surface_pres_Pa', 'T_K', 'windspeed_m_per_s', 'RH', 'value', 'last24h']
index PBL_m
<class 'numpy.ndarray'> (32111, 1)
index Surface_pres_Pa
<class 'numpy.ndarray'> (32111, 1)
index T_K
<class 'numpy.ndarray'> (32111, 1)
index windspeed_m_per_s
<class 'numpy.ndarray'> (32111, 1)
index RH
<class 'numpy.ndarray'> (32111, 1)
index value
<class 'numpy.ndarray'> (32111, 1)
index last24h
<class 'numpy.ndarray'> (32111, 1)
7 ['PBL_m', 'Surface_pres_Pa', 'T_K', 'windspeed_m_per_s', 'RH', 'value', 'last24h']
index PBL_m
<class 'numpy.ndarray'> (33895, 1)
index Surface_pres_Pa
<class 'numpy.ndarray'> (33895, 1)
index T_K
<class 'numpy.ndarray'> (33895, 1)
index windspeed_m_per_s
<class 'numpy.ndarray'> (33895, 1)
index RH
<class 'numpy.ndarray'> (33895, 1)
index value
<class 'numpy.ndarray'> (33895, 1)
index last24h
<class 'numpy.ndarray'> (33895, 1)
7 ['PBL_m', 'Surface_pres_Pa', 'T_K', 'windspeed_m_per_s', 'RH', 'value', 'last24h']
index PBL_m
<class 'numpy.ndarray'> (31299, 1)
in